# 平均報酬強化学習

参考：
* [Average-reward model-free reinforcement learning: a systematic review and literature mapping](https://arxiv.org/abs/2010.08920)
* [Markov Decision Processes: Discrete Stochastic Dynamic Programming](https://onlinelibrary.wiley.com/doi/book/10.1002/9780470316887)
* [Average reward reinforcement learning: Foundations, Algorithms, and Empirical Results](https://link.springer.com/content/pdf/10.1007/BF00114727.pdf)

今回は平均報酬強化学習について学びます．また，最初に無限ホライゾンの問題設定について少し学びます．

## MDPの構造

参考：
* [Average reward reinforcement learning: Foundations, Algorithms, and Empirical Results](https://link.springer.com/content/pdf/10.1007/BF00114727.pdf)
* [Markov Decision Processes: Discrete Stochastic Dynamic Programming](https://onlinelibrary.wiley.com/doi/book/10.1002/9780470316887)
* 

後で見る割引問題では気にしなくても良いですが（多分擬似的なホライゾンができるため？），平均報酬ではMDPの構造を気にしないと困ることがあります．
ここでは，いくつか重要な用語や定義についてまとめます．

### マルコフ連鎖の用語

平均報酬でよく使うので，マルコフ連鎖についての用語をまとめます．MDPでは似たような用語になります．

* $\left\{X_n, n=0,1,2, \ldots\right\}$を離散状態空間$S$に値を取る確率変数とします．$S$は有限もしくは可算です．
* 次を$n \geq 1$かつ$j_k \in S, 0 \leq k \leq n$について満たすとき，$\left\{X_n, n=0,1,2, \ldots\right\}$は**マルコフ**であると言います：$P\left\{X_n=j_n \mid X_{n-1}=j_{n-1}, X_{n-2}=j_{n-2}, \ldots, X_0=j_0\right\}=P\left\{X_n=j_n \mid X_{n-1}=j_{n-1}\right\}$
* $p^m(j\mid s) = P\left\{X_{n+m}=j \mid X_n=s\right\}$を，$m$ステップ後に$s$から$j$に遷移する確率とします．
  * 簡単な計算で示せますが，これは遷移確率行列の$m$乗の$P^m$の要素になります．
* $S$上の実数関数$g(\cdot)$について，$E_s\left\{g\left(X_n\right)\right\} \equiv \sum_{j \in S} g(j) p^n(j \mid s)$を，$X_0=s$から始まって$n$ステップ目の確率変数の期待値とします．
* また，$P_s\left\{X_n \in A\right\} \equiv E_s\left(I_{(A)}\left(X_n\right)\right\}=\sum_{j \in A} p^n(j \mid s)$の表記も導入します．

#### 状態についての用語

**transientとrecurrent**

$s \in S$について，次の変数を導入します：
* $\nu_s$：$s$を訪問する回数（無限時間かつ$s$からスタートして？）
* $\tau_s$：$s$からスタートして，$s$を初めて訪問する時刻

<!-- ここで，$E_s\{\tau_s\} < \infty$のとき，有限ステップ以内に$s$を必ず再訪問することがわかります．そうでないと，期待値を取ると$\infty$になってしまいます．
また，$P_s\left(\tau_s<\infty\right)<1$は$P_s\left(\tau_s=\infty\right)>0$を意味するので，$s$を再訪問しない確率が少しあることがわかります． -->

この変数に対して，次の用語を導入します：
1. $E_s\{\tau_s\} < \infty$（再訪問するステップの期待値が有限）かつ$P_s\left(\tau_s<\infty\right)<1$（$s$を**永遠にに再訪問しない**確率が少しある）：これは矛盾するので，ありえません．
2. $E_s\{\tau_s\} = \infty$（再訪問するステップの期待値が無限）かつ$P_s\left(\tau_s<\infty\right)<1$（$s$を**永遠に再訪問しない**確率が少しある）：このような$s$を**transient**である，と呼びます．
3. $E_s\{\tau_s\} < \infty$（再訪問するステップの期待値が有限）かつ$P_s\left(\tau_s<\infty\right)=1$（$s$に有限ステップで必ず再訪問する）：このような$s$を**positive recurrent**である，と呼びます
4. $E_s\{\tau_s\} = \infty$（再訪問するステップの期待値が無限）かつ$P_s\left(\tau_s<\infty\right)=1$（$s$に有限ステップで必ず再訪問する）：このような$s$を**null recurrent**である，と呼びます．

次のMDPがNull stateの例です（Birth-death chainと呼ばれたりします．[参考](https://www2.isye.gatech.edu/~spyros/courses/IE7201/Spring-22/Discrete-Time%20Birth-Death%20Process.pdf)）

![birth-death](figs/birth-death-chain.png)

一番左の状態$0$に注目しましょう．次の３パターンがありえます．
1. $p < 1/ 2$のとき，右に行く確率が高くなります．よって，$0$を再訪問するステップの期待値が無限かつ再訪問しない確率が少しあるので，これはTransientです．
2. $p > 1 / 2$のとき，左に行く確率が高くなります．よって，$0$を再訪問するステップの期待値が有限かつ$s$に必ず再訪問するので，これはpositive recurrentです
3. $p=1/2$のとき，$0$を再訪問する確率はありますが，一方で，再訪問する期待値は$\infty$です．よってnull recurrentです．

null recurrentとpositive recurrentは区別しないことが多いです（どちらもまとめてrecurrentと呼ぶ）．
また，recurrentな状態は**ergodic**とも呼びます．
次はrecurrentとtransientの必要十分条件です．

* recurrent : $E_s\left\{\nu_s\right\}=\sum_{n=0}^{\infty} p^n(s \mid s)=\infty$
* transient : $E_s\left\{\nu_s\right\}=\sum_{n=0}^{\infty} p^n(s \mid s)<\infty$

**communicateとirreducible**

* $p^n(j \mid s)>0$を満たす$n \geq 0$が存在するとき，$j$は$s$から**accessible**である，と言い，$s\to j$と書きます
  * そうでない場合は**inaccessible**と書きます
* $s \to j$かつ$j \to s$の場合，$s$と$j$は**communicate**である，と言います（つまりどちらかでも行ける場合）
* $C\subseteq S$は，全ての$s \notin C$が$j \in C$から**accessible**ではない場合，$C$はclosedである，と言います．つまり，$C$から外に出ていけない場合ですね．
* また，$C$がclosedであり，かつ$C$のプロパーな部分集合（$C$を含まない部分集合のこと）の全てがclosedではないとき，$C$を**irreducible**と言います．つまり，$C$から外に出られないですが，$C$の中は自由に行き来できます（多分全部がcommunicatingである必要はないです）．
* irreducibleかつclosedな集合であり，一つの状態からなるものを，**absorbing**である，と言います．
* Closedかつirreducibleな集合は，しばしば異なるマルコフ連鎖とみなされます．

**period**

* $p^n(s\mid s) > 0$であるような$n$の最大公約数を$s$の**period**と言います．つまり，$s$に再訪問可能なステップ数を表します．
* $p(s \mid s) > 0$であるような$s$を**aperiodic**である，と言います．つまり，その場に留まる可能性がある状態です．
* **Periodは状態の集合ついての性質です．** 実際，Closedかつirreducibleな集合の状態は全て同じperiodを持ちます．
  * もしperiodが1より大きければ，その集合はperiodicと言います
  * periodが1の場合，その集合はaperiodicと言います
* irreducibleなマルコフ連鎖のperiodが1より大きい場合，periodicな連鎖と言います．同様に，1の場合はaperiodicな連鎖，と言います．

#### 遷移確率の標準形

recurrentな状態の集合は，互いに素なclosed irreducible集合$C_k$に分解することができます（なんで？）．ここで，$k=1, 2, \dots, m$であり，$S$が有限ならば$m$も有限です．このとき，
$$
S=C_1 \cup C_2 \cup \cdots \cup C_m \cup T
$$
と書けます．ここで$T$はどの$C_k$にも属さないtransientな状態集合です．
このとき，遷移確率行列は
$$
P=\left[\begin{array}{cccccc}
P_1 & 0 & 0 & . & . & 0 \\
0 & P_2 & 0 & . & . & 0 \\
. & & . & & & \\
. & & & . & & \\
0 & & & & P_m & 0 \\
Q_1 & Q_2 & \cdot & \cdot & Q_m & Q_{m+1}
\end{array}\right]
$$
として表現できます．これを遷移確率の標準形と呼びます．

ここで，$P_i$は$C_i$内の状態遷移を表し，$Q_i$は$T$から$C_i$への状態遷移を表します．
また，$Q_{m+1}$は$T$内の状態遷移です．


#### マルコフ連鎖の構造

マルコフ連鎖はいくつかのクラスに分類できます

1. 単一のclosedな状態集合からなるとき，マルコフ連鎖をirreducibleである，と言います（Transientな集合は存在しません）．
2. 有限な状態集合について，一つのclosedな状態集合と一つ（もしくは空の）Transientな集合からなるとき，マルコフ連鎖をUnichainである，と言います．
3. そうでない場合はMultichainである，と言います．


## マルコフ連鎖の状態の区別

次のアルゴリズムはマルコフ連鎖において，
* closedかつirreducibleな集合
* transientな状態集合

を見つけることができます．
次の性質を使います：

1. 状態$s$は任意の状態$j \neq s$について$p(s \mid s)>0$ かつ $p(j \mid s)=0$ ならば，absorbing
2. 状態$s$がabsorbingかつ$p(s\mid k) > 0$ならば，$k$はtransient
3. 状態$s$がtransientかつ$p(s\mid k) > 0$ならば，$k$はtransient
4. 状態$i$が$j$とcommunicatesであり，$j$がtransientならば，$i$はtransient．$j$がrecurrentならば$i$もrecurrent．

この性質は「行列$P$について，どこに$0$があり，どこに非$0$があるか？」にしか依存してないことに注意しましょう．よって，$P$の分解のみが重要ならば，$P$は

* $b(j \mid i)=1$ if $p(j \mid i)>0$, and 0 otherwise.

な行列$B$とみなしても問題ありません．

---

さて，状態空間$S=\{1, 2, \dots, N\}$を考えましょう．
以下のアルゴリズムでは

* $L(i)$は状態$i$に割り当てられたラベルを表します
    * recurrent: R 
    * transient: T
    * unlabeled: O

とします．
* $U$をunlabeledな状態の集合
* $S(i)$を「$i$とcommunicatingだと判断された状態の集合」とします．
* $W$をステップ5で構築された行列の行と列のインデックスの集合

とします．

1. 初期化：$S(i)=\{i\}$, $L(i)=O$，$U=S$，$W=S$
2. ... (TODO: 続き)


### MDPの用語

上でやったのは**マルコフ連鎖**の用語です．
こっちはMDPの用語ですね．

---

次を満たすとき，状態$x$と$y$は方策$\pi$について**communicate**である，と言います．
* 方策$\pi$において，$x\to y$および$y \to x$の確率が非ゼロ．

逆に，次を満たすとき，状態$x$は方策$\pi$について**recurrent**である，と言います．
* $\pi$によって$x$から出発してそこに戻る確率が１

つまり，recurrentな状態は$\pi$において永遠に訪問されます．

---

次を満たすとき，状態$x$は**transient**である，と言います．
* 任意の方策について，$x$から出発してそこに戻る確率が$<1$である．

つまり，transientな状態は有限な回数しかそこに戻ってこれません．

---

次を満たす状態の集合$X$は，$\pi$において**ergodic**もしくは**recurrent**である，と言います．
* $X$の中の全ての状態はrecurrentであり，また，お互いにcommunicateである．

もし全ての状態によってergodicな状態集合が形成されれば，そのマルコフ連鎖は**irreducible**（既約）である，と言います．

---

$p_{s, s}^n(\pi)$を，方策$\pi$が$s$から始まって$n$ステップで再び$s$に至る確率とします．
次を状態$s$における方策$\pi$の**period**と呼びます．
* $p^n_{s, s}(\pi) > 0$であるような全ての$n$の最大公約数

---

次を満たすとき，そのMDPは**ergodic**もしくは**recurrent**である，と言います．
* 任意の（決定的かつ定常な）方策が，単一のrecurrentな状態集合をもつ

次を満たすとき，そのMDPは**unichain**である，と言います．
* 任意の方策が，単一のrecurrentな状態集合と(空でも良い)transientな状態集合をもつ

次を満たすとき，そのMDPは**communicating**である，と言います．
* 全ての状態のペアがcommunicateになるような定常な方策が存在する

次を満たすとき，そのMDPは**multichain**である，と言います．
* ２つ以上のrecurrentな状態集合をもつような定常な方策が存在する．

---

recurrentやtransientな状態の例については論文のFigure 2などを参照してください．


## 無限ホライゾンの目的関数について

* [Markov Decision Processes: Discrete Stochastic Dynamic Programming](https://onlinelibrary.wiley.com/doi/book/10.1002/9780470316887)の５章

無限ホライゾンでは，大きく３種類の目的関数を考えることができます．

1. 期待総報酬：$v^\pi(s) \equiv \lim _{N \rightarrow \infty} E_s^\pi\left\{\sum_{t=1}^N r\left(X_t, Y_t\right)\right\}=\lim _{N \rightarrow \infty} v_{N+1}^\pi(s)$
    * これは$+\infty$もしくは$-\infty$になることがあります．
2. 期待割引総報酬：$v_\lambda^\pi(s) \equiv \lim _{N \rightarrow \infty} E_s^\pi\left\{\sum_{t=1}^N \lambda^{t-1} r\left(X_t, Y_t\right)\right\}$
    * これは$\sup \sup |r(s, a)|=M<\infty$ $s \in S \quad a \in A_s$である限り，極限が存在します．
3. 期待平均報酬（ゲインとも呼ばれます）：$g^\pi(s) \equiv \lim _{N \rightarrow \infty} \frac{1}{N} E_s^\pi\left\{\sum_{t=1}^N r\left(X_t, Y_t\right)\right\}=\lim _{N \rightarrow \infty} \frac{1}{N} v_{N+1}^\pi(s)$
    * これは極限が存在しない場合があります．しかし，次のlimsupとliminfは必ず存在します．
    * $g^\pi(s) \equiv \liminf _{N \rightarrow \infty} \frac{1}{N} v_{N+1}^\pi(s), \quad g_{+}^\pi(s) \equiv \limsup _{N \rightarrow \infty} \frac{1}{N} v_{N+1}^\pi(s)$
    * UnichainなMDPでは，任意の方策について，ゲインが状態に非依存であることに注意しましょう．つまり，$\rho^\pi(x)=\rho^\pi(y)=\rho^\pi$です．
        * recurrentな状態は有限回しか訪問されないので，平均を取ると影響が消えます
        * recurrentな状態は永遠に訪問されるので，状態間で平均報酬が変わりません．

期待平均報酬が存在しない例を見てみましょう．
$S=\{1, 2, \dots\}$とし，各$s \in \mathcal{S}$に対して，$p\left(s+1 \mid s, a_s\right)=1, p\left(j \mid s, a_s\right)=0$ for $j \neq s+1$および$r\left(s, a_s\right)=(-1)^{s+1} s$になるような$a_s$が一つだけ存在するとします．

![no-limit](figs/no-limit-MDP.png)

このとき，方策は一つしか存在しません．それを$\pi$としましょう．また，$r_N(s)=0$としましょう．また，$v_1^\pi(1)=0, v_2^\pi(1)=1$とします．このとき，任意の$N > 1$で
$$
v_N^\pi(1)=\left\{\begin{array}{ll}
k & N=2 k \\
-k & N=2 k+1,
\end{array} k=1,2, \ldots\right.
$$
なので，
$$
\lim \sup _{N \rightarrow \infty} v_N^\pi(1)=+\infty \quad \text { and } \quad \liminf _{N \rightarrow \infty} v_N^\pi(1)=-\infty
$$
であるため，$s=1$における期待総報酬は存在しません．また，
$$
\begin{aligned}
&g_{+}^\pi(1)=\limsup _{N \rightarrow \infty} N^{-1} v_{N+1}^\pi(1)=\frac{1}{2}\\
&g_{-}^\pi(1)=\liminf _{N \rightarrow \infty} N^{-1} v_{N+1}^\pi(1)=-\frac{1}{2}
\end{aligned}
$$
なので，$s=1$におけるゲインも存在しません．


#### 1. 期待総報酬問題について

この問題では$v^\pi(s)=\lim _{N \rightarrow \infty} v_N^\pi(s)$を達成する$\pi$について扱います．
しかし，多くの場合，この$v^\pi(s)$は発散することがあります．
そこで，次の問題だけ考えることでこれを回避します：
1. 任意の方策で$\lim _{N \rightarrow \infty} v_N^\pi(s)$が存在する問題
2. 極限が存在し，かつそれが有限であり，また，任意の方策に対して$v^{\pi^*}(s) \geq \limsup _{N \rightarrow \infty} v_N^\pi(s)$を満たすような方策$\pi^\star$が最低でも一つ存在する問題
3. TODO: Use optimality criteria which are sensitive to the rate at which $v^\pi_N(s)$ diverges.

**1が満足される条件**

$$v_{+}^\pi(s) \equiv E_s^*\left\{\sum_{t=1}^{\infty} \max\{r\left(X_t, Y_t\right), 0\}\right\}$$
$$v_{-}^\pi(s) \equiv E_s^*\left\{\sum_{t=1}^{\infty} \max\{-r\left(X_t, Y_t\right), 0\}\right\}$$

とします．そして，
* 任意の$\pi \in \Pi^{\mathrm{HR}}$について，$v_{+}^\pi(s)$ か $v_{-}^\pi(s)$が有限

であれば，

$$
v^\pi(s)=v_{+}^\pi(s)-v_{-}^\pi(s)
$$

となり，極限が存在します．
また，特に次のケースを区別して考えます

1. (Positive bounded model) 任意の$s \in S$について，$r(s, a) \geq 0$な$a \in A_s$が存在かつ$v^\pi_+(s)$が任意の$\pi \in \Pi^{HR}$で有限
    * この仮定のもとでは，任意の方策は非正の報酬の状態に吸収されます（そうでないと価値が有限になりません）
    * 問題の例：
        * [最適停止問題](https://ja.wikipedia.org/wiki/%E6%9C%80%E9%81%A9%E5%81%9C%E6%AD%A2%E5%95%8F%E9%A1%8C)
        * 何らかの状態に到達する確率を最大化したい問題（ギャンブルなど）
        * 何らかの状態に到達するまでの時間を可能な限り伸ばしたい問題（コンピュータゲームなど）
2. (Negative model) 任意の$s \in S$と$a \in A_s$について，$r(s, a) \leq 0$かつ，何らかの$\pi \in \Pi^{HR}$について，$v^\pi(s) > -\infty$
    * 仮定を満たす方策は報酬が０の状態に吸収されます（そうでないと価値が有限になりません）
    * 問題の例：
        * 何らかの状態に到達しない確率を最大化したい問題
        * 何らかの状態に到達するまでの時間を最小化する問題
        * 最適停止問題
3. (Convergent model) 任意の$s \in S$と任意の$\pi \in \Pi^{HR}$について，$v_{+}^\pi(s)$ and $v_{-}^\pi(s)$が有限

ちなみに，Positive bounded modelとNegative modelは符号を**逆転しただけではありません**．
実際，Positive bounded modelでは報酬を最大化する一方で，Negative modelでは総報酬をゼロに近づけることが目標です．

#### 2. 期待割引総報酬問題について

割引問題は経済のモデルなどで便利です．また，かなり充実した理論が存在し，様々な解法が存在します．
よく知られた話ですが，割引率はランダムなホライゾン$\nu$とみなすこともできます．
これは例えば経済モデルにおける工場の破産や生産設備の故障，生物系のモデルにおける生き物の死亡などをモデル化するときに便利です．

$$
v_\nu^\pi(s) \equiv E_s^\pi\left[E_\nu\left\{\sum_{t=1}^\nu r\left(X_t, Y_t\right)\right\}\right]
$$

を，ランダムなホライゾン$\nu$に対しての期待価値とします．ここで，$\nu$が帰化分布に従う場合を考えてみましょう．つまり，

$$
P\{\nu=n\}=(1-\lambda) \lambda^{n-1}, \quad n=1,2, \ldots
$$

です．幾何分布は上で紹介したような「故障のモデル」などを表すときに便利です．
証明はMarkov decision processesのProposition 5.3.1に任せますが，$v_\nu^\pi(s)=v_\lambda^\pi(s)$ for all $s \in S$が成立します．

### 最適性の評価指標

* [Markov Decision Processes: Discrete Stochastic Dynamic Programming](https://onlinelibrary.wiley.com/doi/book/10.1002/9780470316887)の５章
* [Average reward reinforcement learning: Foundations, Algorithms, and Empirical Results](https://link.springer.com/content/pdf/10.1007/BF00114727.pdf)

上で見た目的関数に対応する最適性の指標について見てみましょう．

---

次を満たす$\pi^\star$は**総報酬最適**である，といいます

$$
v^{\pi^*}(s) \geq v^\pi(s) \quad \text { for each } s \in S \text { and all } \pi \in \Pi^{\mathrm{HR}}
$$

この指標は"positive bounded", "negative"，そして"convergent model"に対して適用できます．

また，次を**MDPの価値**，と呼びます

$$
v^*(s) \equiv \sup _{\pi \in I 1^{\mathrm{HR}}} v^\pi(s)
$$

$v^{\pi^*}(s)=v^*(s) \quad$ for all $s \in S$であるならば，最適方策$\pi^* \in \Pi^K(K=$ HR, HD, MR, or MD $)$が存在します．

---

次を満たす方策$\pi^\star$は**割引最適**である，といいます．
割引率$0 \leq \lambda < 1$について，

$$
v_\lambda^{\pi^*}(s) \geq v_\lambda^\pi(s) \quad \text { for each } s \in S \text { and all } \pi \in \Pi^{\mathrm{HR}}
$$

また，割引モデルに置いて，MDPの価値は

$$
v_\lambda^*(s) \equiv \sup _{\pi \in \Pi^{\mathrm{HR}}} v_\lambda^\pi(s)
$$

によって定義され，上と同様に，次を満たすときに割引最適な方策が存在します．

$$
v_\lambda^{\pi *}(s)=v_\lambda^*(s) \quad \text { for all } s \in S
$$

---

次を満たす方策$\pi^\star$は**平均(ゲイン)最適**である，といいます．

$$
g^{\pi^*}(s) \geq g^\pi(s) \quad \text { for each } s \in S \text { and all } \pi \in \Pi^{\mathrm{HR}}
$$

MDPの価値と最適方策の存在性は今までと同様です．

平均報酬の場合は，最適方策が存在しない場合があります（上で見た例など）．
そのような場合は，次を満たす方策のことを最適方策とします．

$$
g_{-\pi^*}^{\pi^*}(s)=\liminf _{N \rightarrow \infty} \frac{1}{N} v_{N+1}^{\pi^*}(s) \geq \limsup _{N \rightarrow \infty} \frac{1}{N} v_{N+1}^\pi(s)=g_{+}^\pi(s)
$$

また，次を満たす方策をそれぞれ上極限，下極限最適方策，と呼びます．

$$
g_{+}^{\pi^*}(s) \geq g_{+}^\pi(s)
\quad
g_{-}^{\pi^*}(s) \geq g_{-}^\pi(s)
$$

---

TODO: Overtaking optimal方策について言及しよう．


## バイアスについて

平均(ゲイン)最適方策の評価では，その途中経過は無視して，最終的に収束したときの報酬についてだけ考えます（途中経過は$\to \infty$で消えます）．
これは問題によってはあまり嬉しくありません．例えば迷路の問題を考えて見ましょう．ゴールすると$+1$，それ以外は$-1$の報酬とします．
このとき，ゴールに至るようなどんな方策も$+1$の平均報酬になり，最適方策の区別がつかなくなります．
一方で迷路のような問題ではなるべく早くゴールすることが望ましいです．

そこで，**バイアス最適**の概念が役に立ちます．

---

次の関数をバイアス関数（もしくはrelative関数）と言います．

$$v_b^\pi(s) = \lim _{N \rightarrow \infty} \frac{1}{N} E_s^\pi\left\{\sum_{t=1}^N r\left(X_t, Y_t\right) - g^\pi(s) \right\}$$

また，次も成立します．

$$v_b^\pi(s) - v_b^\pi(x) = \lim _{N \rightarrow \infty} \frac{1}{N} E_s^\pi\left\{\sum_{t=1}^N r\left(X_t, Y_t\right)\right\} - 
\frac{1}{N} E_x^\pi\left\{\sum_{t=1}^N r\left(X_t, Y_t\right)
\right\}$$

次を満たすような$\pi^\star$を**バイアス最適**（T-optimalとも呼ばれます）である，といいます
* $v_b^{\pi^\star}(s) \geq v_b^\pi(s)$が全ての$\pi$と$s$

---

バイアス関数と割引価値は，[ローラン展開](https://manabitimes.jp/math/2627)を使うことで変形ができます．

まず，割引率$\lambda$の代わりに，$\lambda=(1+\rho)^{-1}$ or $\rho=(1-\lambda) \lambda^{-1}$なる変数$\rho$を使います．
つまり，$1+\rho$が$\lambda$の逆数です．
このとき，

$$
v_\lambda=(I-\lambda P)^{-1} r=(1+\rho)(\rho I+[P-I])^{-1} r
$$

が明らかに成立します．
ここで，ローラン展開を利用すると，つぎが成立します．

$S$が有限とする．$\nu$を$I-P$のうち，絶対値が最小の非ゼロの固有値とする．このとき，任意の$0 < \rho < |\nu|$について，

$$
v_\lambda=(1+\rho)\left[\rho^{-1} y_{-1}+\sum_{n=0}^{\infty} \rho^n y_n\right]
$$

が成立します．ここで，
$$
y_{-1}=P^* r=g, y_0=H_P r=b, \text { and } y_n=(-1)^n H_P^{n+1} r \text { for } n=1,2, \ldots
$$
です．$H_P$は$H_P=C-\lim _{N \rightarrow \infty} \sum_{n=0}^{N-1}\left(P^N-P^*\right)$で定義され，deviation matrixと呼ばれます．

TODO: 証明．

上の補題から，$g$と$b$をゲインとバイアスとすると，$S$が有限かつ報酬が有限ならば，
$$
v_\lambda=(1-\lambda)^{-1} g+b+f(\lambda)
$$
が成立します．ここで$f(\lambda)$は$\lambda \to 1$でゼロに収束するベクトルです．

TODO: 証明

---


### n-割引最適

上の変形の補題を使うと，ゲイン最適，バイアス最適，そして普通の割引最適が一般化できます．
次のn-割引最適を定義しましょう．

---

$n=-1, 0, 1, \dots$について，次を満たす方策$\pi^\star$を$n$-割引最適と呼びます．

任意の$s$と$\pi$について，

$$
\lim _{\lambda \rightarrow 1}(1-\lambda)^{-n}\left(v_\lambda^{\pi^*}(s)-v_\lambda^\pi(s)\right) \geq 0
$$

この$n$-割引最適について，次が成立します．

1. $n=-1$のとき，ゲイン最適と等価．
2. $n=0$のとき，バイアス最適と等価
3. $n=0$かつ$\lambda$を固定すると，割引最適と等価

**証明**

**$n=1$のとき**

$n=-1$のときを考えましょう．つまり，

$$\lim_{\lambda \rightarrow 1}(1-\lambda)\left(v_\lambda^{\pi^*}(s)-v_\lambda^\pi(s)\right) \geq 0$$
です．

このとき，上でやった割引価値とバイアス関数の変形を使うと，
$$
\lim _{\lambda \rightarrow 1}(1-\lambda)\left(\frac{v_g^{\pi^*}(s)-v_g^\pi(s)}{1-\lambda}+v_b^{\pi^*}(s)+f^{\pi^*}(s, \lambda)-v_b^\pi(s)-f^\pi(s, \lambda)\right) \geq 0
$$
です．$f$の部分は$\lambda \to 1$で$0$に収束するので，
$$
\left(v_g^{\pi^*}(s)-v_g^\pi(s)\right) \geq 0 .
$$
が成り立ち，これはゲイン最適と同じです（多分バイアス関数は上界がある？）．

**$n=0$のとき**

$n=0$のときを考えましょう．つまり，
$$\lim_{\lambda \rightarrow 1}\left(v_\lambda^{\pi^*}(s)-v_\lambda^\pi(s)\right) \geq 0$$
です．
このとき，$\lambda \to 1$を取るので，ゲイン最適ではない$\pi$は考える必要がありません（最適方策の割引価値が支配的な項になるので）．

$\pi$がゲイン最適な方策の状況を考えましょう．
上でやった割引価値とゲイン関数の変形を使うと，
$$
\lim _{\lambda \rightarrow 1}\left(\left(\frac{v_g^{\pi^*}(s)}{1-\lambda}+v_b^{\pi^*}(s)+f^{\pi^*}(s, \lambda)\right)-\left(\frac{v_g^\pi(s)}{1-\lambda}+v_b^\pi(s)+f^\pi(s, \lambda)\right)\right) \geq 0
$$
であり，$\pi$はゲイン最適なので，
$$
\lim _{\lambda \rightarrow 1}\left(\left(v_b^{\pi^*}(s)+f^{\pi^*}(s, \lambda)\right)-\left(v_b^\pi(s)+f^\pi(s, \lambda)\right)\right) \geq 0
$$
です．$f$は$0$に収束するので，
$$
\lim _{\lambda \rightarrow 1}v_b^{\pi^*}(s)-v_b^\pi(s) \geq 0
$$
が成立します．

---

実は，$n < m$であるとき，$m$-割引最適方策は$n$-割引最適方策でもあります．
そのため，バイアス最適な方策はゲイン最適でもあります（下の図参照）．

![n-discount-optimal](figs/n-discount-optimal.png)

全ての$n \geq -1$で最適な方策は**Blackwell最適**とも呼ばれます．
Blackwell最適な方策は次の方策と等価です．
任意の$s$と$\pi$について，次を満たす$\lambda^\star(s)$が存在する．

$$
v_\lambda^{\pi^*}(s)-v_\lambda^\pi(s) \geq 0 \quad \forall \lambda^\star(s) \leq \lambda < 1
$$

つまり，Blackwell最適は割引最適でもあります．また，特に$\sup_{s \in S}\lambda^\star <1$のとき，strongly Blackwell最適とも呼びます．


今回は$n=0, -1$のときだけ考えましょう．

## 定常分布について

* [Average-reward model-free reinforcement learning: a systematic review and literature mapping](https://arxiv.org/abs/2010.08920)も参考

**マルコフ連鎖についての定常分布**

ここでの結果は有限と可算集合のどちらのマルコフ連鎖についても成立します．

$\left\{A_n: n \geq 0\right\}$を行列の系列とします．
そして，$\lim _{n \rightarrow \infty} A_n(j \mid s)=A(j\mid s)$ならば，$\lim _{n \rightarrow \infty} A_n=A$であると表記します．
periodicなマルコフ連鎖については，このような極限が存在しない場合があります．
例えば
$$
P=\left[\begin{array}{ll}
0 & 1 \\
1 & 0
\end{array}\right]
$$

を考えると，$2n$のときは$I$であり，$2n+1$は$P$になるので，極限が存在しません．変わりに，次のCesaro-limitを考えることができます．

$$
\lim _{N \rightarrow \infty} \frac{1}{N} \sum_{n=0}^{N-1} A_n=A,
$$

これは省略して
$$
C-\lim _{N \rightarrow \infty} A_N=A
$$
と書くこともあります．
極限行列を次のように定義しましょう：

$$
P^*=C \underset{N \rightarrow \infty}{-\lim P^N}
$$

このCesaro-limitは$S$が有限であれば必ず存在します（また，それは確率行列になります）．
可算状態の連鎖の場合は確率行列にならないこともあります．
例えば$p(s+1 \mid s)=1$ for $s=0,1, \ldots$の場合は，$P^*$の要素はゼロになります．

ちなみにCesaro-limitではなく，普通の極限$\lim _{N \rightarrow \infty} p^N(j \mid s)$は，irreducibleかつaperiodicなマルコフ連鎖では存在します．

$P$をpositive recurrent irreducibleな連鎖のとき，
$$
q^T = q^T P
$$
はユニークな解を持ちます（ここで$q$は和が１のベクトルです）．
この解のことを$P$についての**定常分布**と呼びます．


**MDPついての定常分布**

次を定義します：
* 方策から誘導されるマルコフ連鎖：$\boldsymbol{P}_\pi \in[0,1]^{|\mathcal{S}| \times|\mathcal{S}|}$
    * その$t$回繰り返したもの：$\boldsymbol{P}_\pi^t\left[s_0, s\right]=\operatorname{Pr}\left\{S_t=s \mid s_0, \pi\right\}=: p_\pi^t\left(s \mid s_0\right)$

このとき，このマルコフ連鎖の収束先は
$$p_\pi^{\star}\left(s \mid s_0\right)=\lim _{t_{\max } \rightarrow \infty} \frac{1}{t_{\max }} \sum_{t=0}^{t_{\max }-1} p_\pi^t\left(s \mid s_0\right)=\lim _{t_{\max } \rightarrow \infty} p_\pi^{t_{\max }}\left(s \mid s_0\right), \quad \forall s \in \mathcal{S}$$

を満たします．この最初のlimitは有限なMDPならば必ず存在し，２つ目のLimitはMDPがaperiodicであれば存在します（TODO: 証明）

また，$p_\pi^\star(s|s_0)$をベクトル化したものを$\boldsymbol{p}_\pi^{\star} \in[0,1]^{|\mathcal{S}|}$とします．
これは
$$\left(\boldsymbol{p}_\pi^{\star}\right)^{\top} \boldsymbol{P}_\pi=\left(\boldsymbol{p}_\pi^{\star}\right)^{\top}$$
を満たします

## ゲインについて

平均報酬はゲインとも呼ばれ，次で定義され，以下の変形が成立します：

$$
\begin{aligned}
v_g\left(\pi, s_0\right) & :=\lim _{t_{\max } \rightarrow \infty} \frac{1}{t_{\max }} \mathbb{E}_{S_t, A_t}\left[\sum_{t=0}^{t_{\max }-1} r\left(S_t, A_t\right) \mid S_0=s_0, \pi\right] \\
& =\sum_{s \in \mathcal{S}}\left\{\lim _{t_{\max } \rightarrow \infty} \frac{1}{t_{\max }} \sum_{t=0}^{t_{\max }-1} p_\pi^t\left(s \mid s_0\right)\right\} r_\pi(s)=\sum_{s \in \mathcal{S}} p_\pi^{\star}\left(s \mid s_0\right) r_\pi(s) \\
& =\lim _{t_{\max } \rightarrow \infty} \mathbb{E}_{S_{t_{\max }}, A_{t_{\max }}}\left[r\left(S_{t_{\max }}, A_{t_{\max }}\right) \mid S_0=s_0, \pi\right]
\end{aligned}
$$

ここで，
* $r_\pi(s)=\sum_{a \in \mathcal{A}} \pi(a \mid s) r(s, a)$
* 最初のlimitは方策が定常かつMDPが有限であれば存在します（TODO: 証明）
* ２つ目のLimitは簡単です
* ３つ目は方策が定常かつMDPがaperiodic，もしくは報酬に何らかの構造が入っている場合に存在します（TODO: 証明）

行列形式で書けば，

$$
\boldsymbol{v}_g(\pi)=\lim _{t_{\max } \rightarrow \infty} \frac{1}{t_{\max }} \boldsymbol{v}_{t_{\max }}(\pi)=\lim _{t_{\max } \rightarrow \infty} \frac{1}{t_{\max }} \sum_{t=0}^{t_{\max }-1} \boldsymbol{P}_\pi^t \boldsymbol{r}_\pi=\boldsymbol{P}_\pi^{\star} \boldsymbol{r}_\pi
$$

です．ここで，$\boldsymbol{r}_\pi \in \mathbb{R}^{|\mathcal{S}|}$ は $r_\pi(s)$をstackしたものです．

定常分布は$s_0$に依存しないので，$\boldsymbol{P}_\pi^{\star}$の行は全て同じであることに気をつけましょう．つまり，

* $v_g(\pi)=\boldsymbol{p}_\pi^{\star} \cdot \boldsymbol{r}_\pi=v_g\left(\pi, s_0\right), \forall s_0 \in \mathcal{S}$
* $\boldsymbol{v}_g(\pi)=v_g(\pi) \cdot \mathbf{1}$

です．

方策$\pi^* \in \Pi_{\mathrm{S}}$は次を満たす時ゲイン最適であると呼びます：

$$
v_g\left(\pi^*, s_0\right) \geq v_g(\pi, s_0), \quad \forall \pi \in \Pi_{\mathrm{S}}, \forall s_0 \in \mathcal{S}, \quad \text { hence } \pi^* \in \underset{\pi \in \Pi_{\mathrm{S}}}{\operatorname{argmax}}\; v_g(\pi)
$$


## ベルマン方程式

平均報酬についても以下のベルマン方程式が成立します：

$$
v_b\left(\pi^*, s\right)+v_g\left(\pi^*\right)=\max _{a \in \mathcal{A}}\left\{r(s, a)+\sum_{c^{\prime} \in \mathcal{S}} p\left(s^{\prime} \mid s, a\right) v_b\left(\pi^*, s^{\prime}\right)\right\}, \quad \forall s \in \mathcal{S},
$$

こおｋで，$v_b$はバイアスと呼ばれ，次で定義されます：

$$
\begin{aligned}
v_b\left(\pi, s_0\right) & :=\lim _{t_{\max } \rightarrow \infty} \mathbb{E}_{S_t, A_t}\left[\sum_{t=0}^{t_{\max }-1}\left(r\left(S_t, A_t\right)-v_g(\pi)\right) \mid S_0=s_0, \pi\right] \\
& =\lim _{t_{\max } \rightarrow \infty} \sum_{t=0}^{t_{\max }-1} \sum_{s \in \mathcal{S}}\left(p_\pi^t\left(s \mid s_0\right)-p_\pi^{\star}(s)\right) r_\pi(s) \\
& =\underbrace{\sum_{t=0}^{\tau-1} \sum_{s \in \mathcal{S}} p_\pi^t\left(s \mid s_0\right) r_\pi(s)}_{\text {the expected total reward } v_\tau}-\tau v_g(\pi)+\underbrace{\lim _{t_{\max } \rightarrow \infty} \sum_{t=\tau}^{t_{\max }-1} \sum_{s \in \mathcal{S}}\left(p_\pi^t\left(s \mid s_0\right)-p_\pi^{\star}(s)\right) r_\pi(s)}_{\text {approaches } 0 \text { as } \tau \rightarrow \infty} \\
& =\sum_{s \in \mathcal{S}}\left\{\lim _{t_{\max } \rightarrow \infty} \sum_{t=0}^{t_{\max }-1}\left(p_\pi^t\left(s \mid s_0\right)-p_\pi^{\star}(s)\right)\right\} r_\pi(s)=\sum_{s \in \mathcal{S}} d_\pi\left(s \mid s_0\right) r_\pi(s),
\end{aligned}
$$

ここで，$d_\pi(s|s_0)$は行列$\boldsymbol{D}_\pi:=\left(\boldsymbol{I}-\boldsymbol{P}_\pi+\boldsymbol{P}_\pi^{\star}\right)^{-1}\left(\boldsymbol{I}-\boldsymbol{P}_\pi^{\star}\right)$と同値です（limitと期待値を入れ替えて良い場合）．

バイアスは以下の解釈ができます：

1. （１行目）$s_0$から始まって$\pi$に従うときの，即時報酬$r(s_t, a_t)$と定常報酬$v_g(\pi)$の差
2. （２行目）$s_0$から始まった過程と，$p_\pi^\star$からサンプルされた初期状態から始まった過程の，報酬の差．よって，$\pi$によって得られる総報酬と，ずっと$v_g(\pi)$の報酬の場合の総報酬の差．
3. （３行目）$v_\tau\left(\pi, s_0\right) \approx v_g(\pi) \tau+v_b\left(\pi, s_0\right)$とみなせるので，期待総報酬$v_\tau$がどれくらい振動するかを表している．
4. （４行目）$\left(p_\pi^t\left(s \mid s_0\right)-p_\pi^{\star}(s)\right)$は定常状態になるまで非ゼロなので，バイアスは一時的な性能とみなせる．

何らかの参照状態$s_{\mathrm{ref}} \in \mathcal{S}$について，相対価値$v_{\text {brel }}$が定義されます：

$$
v_{b r e l}(\pi, s):=v_b(\pi, s)-v_b\left(\pi, s_{\text {ref }}\right)=\lim _{t_{\max } \rightarrow \infty}\left\{v_{t_{\max }}(\pi, s)-v_{t_{\max }}\left(\pi, s_{\text {ref }}\right)\right\}, \quad \forall \pi \in \Pi_{\mathrm{S}}, \forall s \in \mathcal{S} .
$$


## 価値反復法

ベルマン方程式より，$\boldsymbol{v}_b^* \in \mathbb{R}^{\mid \mathcal{S}\mid}$がわかれば最適方策も求まります（このとき，最適ゲイン$v_g^*$も必要になります）．
そのため，目標は$\boldsymbol{v}_b^* \in \mathbb{R}^{\mid \mathcal{S}\mid}$を求めればよいです．
通常の動的計画法と同様にして，$\boldsymbol{v}_b^*$の代わりに$\boldsymbol{q}_b^*\in \mathbb{R}^{|\mathcal{S}||\mathcal{A}|}$を求めます．

---

**状態価値関数によるアプローチ**

1. $\hat{v}_b^{k=0}(s) \leftarrow 0$として初期化します
2. 適当な固定された参照状態$s_{\text{ref }}$を使って，次のベルマン作用素で更新します：$\hat{v}_b^{k+1}(s)=\mathbb{B}^*\left[\hat{v}_b^k(s)\right]-\hat{v}_b^k\left(s_{\text {ref }}\right), \quad \forall s \in \mathcal{S}$
$$
\mathbb{B}^*\left[\hat{v}_b^k(s)\right]:=\max _{a \in \mathcal{A}}\left[r(s, a)+\sum_{s^{\prime} \in \mathcal{S}} p\left(s^{\prime} \mid s, a\right) \hat{v}_b^k\left(s^{\prime}\right)\right], \quad \forall s \in \mathcal{S} .
$$
3. スパン関数を$s p(\boldsymbol{v}):=\max _{s \in \mathcal{S}} v(s)-\min _{s^{\prime} \in \mathcal{S}} v\left(s^{\prime}\right)$とします．$s p\left(\hat{\boldsymbol{v}}_b^{k+1}-\hat{\boldsymbol{v}}_b^k\right)\leq\varepsilon$ であればアルゴリズムを停止します．

このアルゴリズムは$k\to \infty$で$\hat{v}_b^k\left(s_{\text {ref }}\right)$が$v_g^*$に収束することが知られています．

---

**行動価値関数によるアプローチ**

行動価値関数を
$$
\begin{aligned}
q_b(\pi, s, a) & :=\lim _{t_{\max } \rightarrow \infty} \mathbb{E}_{S_t, A_t}\left[\sum_{t=0}^{t_{\max }-1}\left(r\left(S_t, A_t\right)-v_g^\pi\right) \mid S_0=s, A_0=a, \pi\right] \\
& =r(s, a)-v_g^\pi+\mathbb{E}\left[v_b^\pi\left(S_{t+1}\right)\right], \quad \forall(s, a) \in \mathcal{S} \times \mathcal{A},
\end{aligned}
$$
で定義します．このとき，次のベルマン方程式が成立します．

$$
q_b^*(s, a)+v_g^*=r(s, a)+\sum_{s^{\prime} \in \mathcal{S}} p\left(s^{\prime} \mid s, a\right) \underbrace{\max _{a^{\prime} \in \mathcal{A}} q_b^*\left(s^{\prime}, a^{\prime}\right)}_{v_b^*\left(s^{\prime}\right)}, \quad \forall(s, a) \in \mathcal{S} \times \mathcal{A},
$$

これを使って，次によって$q$を更新することを考えます．

$$
\begin{aligned}
\hat{q}_b^{k+1}(s, a)&=\mathbb{B}^*\left[\hat{q}_b^k(s, a)\right]-v_b^k\left(s_{\text {ref }}\right)\\
&=\underbrace{r(s, a)+\sum_{s^{\prime} \in \mathcal{S}} p\left(s^{\prime} \mid s, a\right) \max _{a^{\prime} \in \mathcal{A}} \hat{q}_b^k\left(s^{\prime}, a^{\prime}\right)}_{\mathbb{B}^*\left[\hat{q}_b^k(s, a)\right]}-\underbrace{\max _{a^{\prime \prime} \in \mathcal{A}} \hat{q}_b^k\left(s_{\text {ref }}, a^{\prime \prime}\right)}_{\text {can be interpreted as } \hat{v}_g^*},
\end{aligned}
$$

この反復は$\hat{v}_b^k(s)=\max _{a \in \mathcal{A}} \hat{q}_b^k(s, a)$が$v_b^*(s)$に収束することが知られてます．

---

**方策評価**

次の方程式を解くことで方策評価ができます：（TODO: どうやって解くの？）

$$
v_b^k(s)+v_g^k=r(s, a)+\sum_{s^{\prime} \in \mathcal{S}} p\left(s^{\prime} \mid s, a\right) v_b^k\left(s^{\prime}\right), \quad \forall s \in \mathcal{S}, \text { with } a=\hat{\pi}^k(s)
$$

In [2]:
import numpy as np
import jax.numpy as jnp
from jax.random import PRNGKey
import jax
from typing import NamedTuple, Optional

key = PRNGKey(0)

S = 10  # 状態集合のサイズ
A = 3  # 行動集合のサイズ
S_set = jnp.arange(S)  # 状態集合
A_set = jnp.arange(A)  # 行動集合
gamma = 0.8  # 割引率


# 報酬行列を適当に作ります
key, _ = jax.random.split(key)
rew = jax.random.uniform(key=key, shape=(S, A))
assert rew.shape == (S, A)


# 遷移確率行列を適当に作ります
key, _ = jax.random.split(key)
P = jax.random.uniform(key=key, shape=(S*A, S))
P = P / jnp.sum(P, axis=-1, keepdims=True)  # 正規化して確率にします
P = P.reshape(S, A, S)
np.testing.assert_allclose(P.sum(axis=-1), 1, atol=1e-6)  # ちゃんと確率行列になっているか確認します


# 状態集合, 行動集合, 割引率, 報酬行列, 遷移確率行列が準備できたのでMDPのクラスを作ります

class MDP(NamedTuple):
    S_set: jnp.array  # 状態集合
    A_set: jnp.array  # 行動集合
    rew: jnp.array  # 報酬行列
    P: jnp.array  # 遷移確率行列

    @property
    def S(self) -> int:  # 状態空間のサイズ
        return len(self.S_set)

    @property
    def A(self) -> int:  # 行動空間のサイズ
        return len(self.A_set)


mdp = MDP(S_set, A_set, rew, P)

print("状態数：", mdp.S)
print("行動数：", mdp.A)

状態数： 10
行動数： 3


In [10]:

ref_state = 0
Q = jnp.zeros((S, A))

for i in range(20):
    next_v = mdp.P @ Q.max(axis=1)
    nQ = mdp.rew + next_v - Q[ref_state].max()
    span = (nQ.max(axis=1) - Q.max(axis=1)).max() - (nQ.max(axis=1) - Q.max(axis=1)).min()
    Q = nQ
    print(span)

0.58395743
0.12138826
0.026543736
0.007195711
0.0011954308
0.0003478527
5.9604645e-05
1.2993813e-05
2.861023e-06
3.5762787e-07
2.3841858e-07
1.1920929e-07
1.1920929e-07
2.3841858e-07
2.3841858e-07
2.3841858e-07
2.3841858e-07
2.3841858e-07
2.3841858e-07
2.3841858e-07
